Where the actual experiment happens!

In [ ]:

#do necessary imports
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys
import torch
import json
import torch.nn as nn
import torch.optim as optim
from rean.utils import make_run_dir, to_serializable
from rean.data.Dataset import make_datasets
from rean.models.CNN import PlainCNN
from rean.models.P4 import P4CNN
from rean.models.RelaxedP4 import RelaxedP4CNN
from rean.train import train_full, evaluate
from rean.plot import LossPlot
import matplotlib.pyplot as plt


In [ ]:
#some params that should be the same across all experimental runs

group_order = 4
hidden_dim = 8
out_channels = hidden_dim
classes = 10
kernel_size = 3
num_gconvs = 4
num_epochs = 2
batch_size = 64
learning_rate = 0.001
gamma = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Experiment Running Loop #

In [ ]:
#test full loop
models = ['PlainCNN', 'P4CNN', 'RelaxedP4CNN']
noise_types = ['none']
stds = [0.1, 0.2]

for model_name in models:
    for noise_type in noise_types:
        for i, std in enumerate(stds):
            if noise_type == 'none':
                if i != 0: #only run noiseless once
                    continue
                train_noise = None
                noise_params = {"mean":0, "std":0, 'gamma':0}

            else:
                train_noise = noise_type
                test_noise = noise_type #could be none?
                noise_params = {'mean': 0, 'std': std, 'gamma': gamma}
            #make datasets
            train_ds, val_ds, test_ds, in_channels = make_datasets(train_noise=train_noise,
                                                            noise_params=noise_params,
                                                            group_order=group_order)
            #for now to make faster:
            train_ds = torch.utils.data.Subset(train_ds, range(0, 1000))
            #train model
            run_data, best_model = train_full(model_name=model_name,
                          train_ds=train_ds,
                          val_ds=val_ds,
                          in_channels=in_channels,
                          hidden_dim=hidden_dim,
                          out_channels=out_channels,
                          classes=classes,
                          num_gconvs=num_gconvs,
                          kernel_size=kernel_size,
                          group_order=group_order,
                          num_epochs=num_epochs,
                          batch_size=batch_size,
                          learning_rate=learning_rate,
                          device=device)
            run_data['noise_type'] = noise_type
            run_data['std'] = std

            run_dir = make_run_dir(model_name, noise_type, noise_params = noise_params, learning_rate = learning_rate)
            with open(run_dir / "run_data.json", "w") as f:
                json.dump(run_data, f, default = to_serializable, indent=2)

            #test the model
            test_loader = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, pin_memory =True)
            criterion = nn.CrossEntropyLoss()
            test_loss, test_acc = evaluate(best_model,  device, test_loader, criterion = criterion)
            test = {
                "test_loss": test_loss,
                "test_acc": test_acc
            }
            print(f"Test Loss: {test_loss}, Test Acc: {test_acc}")
            with open(run_dir / "test_data.json", "w") as f:
                json.dump(test, f, default = to_serializable, indent=2)



# Plotting #


In [ ]:
#first, plot the performance of all 3 models on the baseline test set, on two sets of axes: one for train, one for validation
fig, (train_ax, val_ax) = plt.subplots(1,2)
runs = []
tests = []
for model_name in models:
    runpath = Path(f"./runs/{model_name}_none_std0_gamma0_lr{learning_rate}")

    #load the rundata in from the json
    rundatapath = runpath / "run_data.json"
    testdatapath = runpath / "test_data.json"
    with rundatapath.open("r", encoding="utf-8") as f:
        run_data = json.load(f)
    with testdatapath.open("r", encoding="utf-8") as f:
        test_data = json.load(f)
    run_data.update(test_data)
    runs.append(run_data)
    tests.append(test_data)

#make loss plots
train_ax = LossPlot(train_ax, runs, labels = ["model_name"], title = "Training Loss", val = False)
val_ax = LossPlot(val_ax, runs, labels = ['model_name'], title = "Validation Loss", train = False)
fig.tight_layout()
fig.suptitle("Training and Validation Loss, Baseline (No Noise) Experiments", y=1.02)
fig.savefig("baseline_loss_plots.png")







In [ ]:
# do the same for each noise type, but plotting both std runs on the same axes
#so, each axes object will have 6 lines: 3 models x 2 stds
noise_types = ['iso', 'aniso']
for noise_type in noise_types:
    fig, (train_ax, val_ax) = plt.subplots(1,2)
    runs = []
    tests = []
    for model_name in models:
        for std in stds:
            runpath = Path(f"./runs/{model_name}_{noise_type}_std{std}_gamma{gamma}_lr{learning_rate}")

            #load the rundata in from the json
            rundatapath = runpath / "run_data.json"
            testdatapath = runpath / "test_data.json"
            with rundatapath.open("r", encoding="utf-8") as f:
                run_data = json.load(f)
            with testdatapath.open("r", encoding="utf-8") as f:
                test_data = json.load(f)
            run_data.update(test_data)
            runs.append(run_data)
            tests.append(test_data)

    #make loss plots
    train_ax = LossPlot(train_ax, runs, labels = ["model_name", "std"], title = "Training Loss", val = False)
    val_ax = LossPlot(val_ax, runs, labels = ['model_name', "std"], title = "Validation Loss", train = False)
    fig.tight_layout()
    fig.suptitle(f"Training and Validation Loss, {noise_type.capitalize()} Noise Experiments", y=1.02)
    fig.savefig(f"{noise_type}_loss_plots.png")


In [ ]:
#fianlly, generate a dataframe for test accuracies across all runs
import pandas as pd
all_runs = []
for model_name in models:
    for noise_type in noise_types + ['none']:
        if noise_type == 'none':
            runpath = Path(f"./runs/{model_name}_none_std0_gamma0_lr{learning_rate}")
            std = 0
        for std in stds:
            runpath = Path(f"./runs/{model_name}_{noise_type}_std{std}_gamma{gamma}_lr{learning_rate}")
            #load the rundata in from the json
            rundatapath = runpath / "run_data.json"
            testdatapath = runpath / "test_data.json"
            with rundatapath.open("r", encoding="utf-8") as f:
                run_data = json.load(f)
            with testdatapath.open("r", encoding="utf-8") as f:
                test_data = json.load(f)
            run_data.update(test_data)
            all_runs.append(run_data)
df = pd.DataFrame(all_runs)
df = df[['model_name', 'noise_type', 'std', 'test_acc']] #get only the stuff we want
df.to_csv("test_accuracies.csv", index=False)